In [ ]:
import json
import numpy as np
import pandas as pd
from keras.models import load_model
import backtrader as bt

import sys
import os
sys.path.append(os.path.abspath("../.."))

In [ ]:
# first import the data again 
df_raw = pd.read_csv('../../data/processed-data/g_2010_2023.csv', index_col="Date", parse_dates=["Date"])
print(df_raw.head())


import joblib
scaler = joblib.load('./scaler.pkl')
from scripts.process import process

from collections import Counter
X, Y = process(df_raw, scaler, dev=False)

# 1) Load your saved model
model = load_model('./regulized_model.keras')
proba   = model.predict(X)             
pred_ix = np.argmax(proba, axis=1)
class_map = {0: 'sell', 1: 'hold', 2: 'buy'}
pred_labels = [class_map[i] for i in pred_ix]


# lets count how many buy signals we got 
counts = Counter(pred_labels)
print("Counts:", counts)

             close    open    high     low    volume
Date                                                
2010-01-04  1118.3  1099.0  1124.6  1093.8  155480.0
2010-01-05  1118.7  1122.0  1129.6  1115.5  156410.0
2010-01-06  1136.5  1118.5  1141.0  1116.8  176900.0
2010-01-07  1133.7  1139.0  1139.5  1128.7  121150.0
2010-01-08  1138.9  1131.5  1140.0  1119.5  212620.0


2025-06-08 21:30:48.101542: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


121/121 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step
Counts: Counter({'hold': 1870, 'buy': 1115, 'sell': 870})


In [28]:
class TestStrategy(bt.Strategy):
    params = dict(
        atr_period=14,
        atr_multiplier=1.0,     # for trailing stop
        risk_per_trade=0.01     # fraction of equity
    )

    def log(self, txt, dt=None):
        """Logging helper."""
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt.isoformat()}, {txt} (Equity: {self.broker.getvalue():.2f})")

    def __init__(self):
        self.atr = bt.ind.ATR(self.data, period=self.p.atr_period)
        self.order = None
        self.position_type = None
        self.trade_log = []  # will hold dicts of trade info
        self.prediction = []
        

    def notify_order(self, order):
        if order.status in (order.Submitted, order.Accepted):
            return

        if order.status == order.Completed:
            side = 'buy' if order.isbuy() else 'sell'
            price = order.executed.price
            self.log(f"{side.upper()} EXECUTED @ {price:.2f}")
            self.position_type = side
            # record entry for trade logger
            self._current_trade = {
                'entry_date': bt.num2date(order.executed.dt).date().isoformat(),
                'type': side,
                'size': order.executed.size,
                'entry_price': price
            }
        elif order.status in (order.Canceled, order.Margin, order.Rejected):
            self.log(f"Order {order.getstatusname()}")

        self.order = None
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        # Get raw close datetime
        raw_dt = trade.close_datetime()

        # If it's already a datetime, just extract the date
        if isinstance(raw_dt, datetime.datetime):
            exit_date = raw_dt.date().isoformat()
        else:
            # Otherwise treat it as an ordinal float
            exit_date = bt.num2date(raw_dt).date().isoformat()

        exit_price = trade.price
        pnl        = trade.pnl

        # Finalize your current trade record
        self._current_trade.update({
            'exit_date':  exit_date,
            'exit_price': exit_price,
            'pnl':        pnl
        })
        self.trade_log.append(self._current_trade)
        self.log(f"TRADE CLOSED PnL: {pnl:.2f}")


    def next(self):
        # warm-up & bounds check
        n = 34 + 20 + 1
        if len(self) <= n or len(self) > len(pred_labels) + n:
            return

        prediction = pred_labels[len(self) - n - 1]
        self.prediction.append({
                                "date": self.data.datetime.date(0).isoformat(),
                                "prediction": prediction
                                })
        if self.order:
            return  # pending order

        if not self.position:
            # open new
            if prediction in ('buy', 'sell'):
                self._place_order(prediction)
        else:
            # flip position
            if prediction != self.position_type:
                self.log(f"FLIP {self.position_type.upper()} → {prediction.upper()}")
                self.close()
                self._place_order(prediction)

    def _place_order(self, side):
        atr_dist = self.atr[0] * self.p.atr_multiplier
        capital = self.broker.getvalue()
        size = (capital * self.p.risk_per_trade) / atr_dist

        if side == 'buy':
            self.buy(size=size, exectype=bt.Order.StopTrail, trailamount=atr_dist)
        else:
            self.sell(size=size, exectype=bt.Order.StopTrail, trailamount=atr_dist)

    def stop(self):
        # write out logs at end
        df = pd.DataFrame(self.trade_log)
        df.to_csv('trade_log.csv', index=False)
        
        df = pd.DataFrame(self.prediction)
        df.to_csv('predictions.csv', index=False)
        
        with open('trade_log.json', 'w') as f:
            json.dump(self.trade_log, f, indent=2)
        
            
        print(f"\nSaved {len(self.trade_log)} trades to trade_log.csv / .json")


In [29]:
import backtrader.analyzers as btanalyzers

# Create a cerebro entity
cerebro = bt.Cerebro()
# Add a strategy
cerebro.addstrategy(TestStrategy)
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='trades')
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(btanalyzers.SQN, _name='sqn')
cerebro.addanalyzer(btanalyzers.Returns, _name='returns')

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
datapath = '../../data/processed-data/g_2010_2023.csv'

# Create a Data Feed# Add a strategy
import backtrader.feeds as btfeeds

data = btfeeds.GenericCSVData(
    dataname=datapath,
    dtformat=('%Y-%m-%d'),
    
    datetime=0,
    time=-1,
    openinterest=-1,
    high=3,
    low=4,
    close=1,
    volume=5,
    open=2,
    # nullvalue=0.0
) # type: ignore
# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(10000.0)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
strats = cerebro.run()
import matplotlib.pyplot as plt

figs = cerebro.plot()
plt.show()

strat = strats[0]


print("Total Trades:", strat.analyzers.trades.get_analysis()['total']['total'])
print("Net Profit:", strat.analyzers.trades.get_analysis()['pnl']['net']['total'])
print("Sharpe Ratio:", strat.analyzers.sharpe.get_analysis())
print("Max Drawdown %:", strat.analyzers.drawdown.get_analysis()['max']['drawdown'])
print("Max Drawdown Value:", strat.analyzers.drawdown.get_analysis()['max']['moneydown'])
print("SQN:", strat.analyzers.sqn.get_analysis())
print("Returns:", strat.analyzers.returns.get_analysis()['rtot'])


# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2010-04-01, BUY EXECUTED @ 1123.73 (Equity: 10013.16)
2010-04-01, Order Margin (Equity: 10013.16)
2010-04-05, Order Margin (Equity: 10055.87)
2010-04-12, FLIP BUY → HOLD (Equity: 10213.41)
2010-04-13, SELL EXECUTED @ 1156.20 (Equity: 10131.78)
2010-04-13, SELL EXECUTED @ 1145.49 (Equity: 10131.78)
2010-04-13, TRADE CLOSED PnL: 180.13 (Equity: 10131.78)
2010-04-13, FLIP SELL → HOLD (Equity: 10131.78)
2010-04-14, BUY EXECUTED @ 1152.00 (Equity: 10140.34)
2010-04-14, TRADE CLOSED PnL: -39.79 (Equity: 10140.34)
2010-04-16, SELL EXECUTED @ 1143.58 (Equity: 10271.12)
2010-04-16, SELL EXECUTED @ 1143.94 (Equity: 10271.12)
2010-04-16, SELL EXECUTED @ 1144.28 (Equity: 10271.12)
2010-04-16, FLIP SELL → HOLD (Equity: 10271.12)
2010-04-19, BUY EXECUTED @ 1137.00 (Equity: 10269.26)
2010-04-19, TRADE CLOSED PnL: 128.92 (Equity: 10269.26)
2010-04-22, SELL EXECUTED @ 1132.19 (Equity: 10203.05)
2010-04-22, FLIP SELL → BUY (Equity: 10203.05)
2010-04-23, BUY EXECUTED @ 

<IPython.core.display.Javascript object>

Total Trades: 666
Net Profit: 4774755.734716963
Sharpe Ratio: OrderedDict([('sharperatio', 2.355893848078167)])
Max Drawdown %: 9.162175375801727
Max Drawdown Value: 131089.6119887773
SQN: AutoOrderedDict([('sqn', 5.775727455630069), ('trades', 666)])
Returns: 6.170605161463982
Final Portfolio Value: 4784755.73


In [21]:
cerebro.plot()[0][0].show()

<IPython.core.display.Javascript object>

In [24]:
%matplotlib inline

<IPython.core.display.Javascript object>

In [36]:
# read the predictions
predictions_df = pd.read_csv('predictions.csv')

predictions_df[['proba_sell', 'proba_hold', 'proba_buy']] = proba

In [39]:
predictions_df.head()

# save the predictions to a csv file
predictions_df.to_csv('predictions_with_proba.csv', index=False)

# save the predictions to a json file
predictions_df.to_json('predictions_with_proba.json', orient='records', date_format='iso')